<a href="https://colab.research.google.com/github/manish2021iitd/Deep-Learning/blob/main/DLassignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [2]:
import wandb

In [ ]:
wandb.login(key='e3c892d4f8c9cd9b9043d31938ad090f0a32cec1')

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
torch.cuda.is_available()

In [7]:
tranaform = transforms.ToTensor()


In [ ]:
import torch
import torch.nn as nn
#class of convolution neural network
class CNN(nn.Module):
    #intitialer for parameters
    def __init__(self, num_filters=32, filter_size=3, num_classes=10):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=num_filters, kernel_size=filter_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=num_filters, out_channels=num_filters*2, kernel_size=filter_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=num_filters*2, out_channels=num_filters*4, kernel_size=filter_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=num_filters*4, out_channels=num_filters*8, kernel_size=filter_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=num_filters*8, out_channels=num_filters*16, kernel_size=filter_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.dense_layers = nn.Sequential(
            nn.Linear(num_filters*16*7*7, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )

    #forward pass
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.dense_layers(x)
        return x


model = CNN()
print(model)
